![Ethereum Wallet loading screen](https://i.imgur.com/BDs8lGK.png)

# ETHPrize dev interviews analysis

Using manually tagged data

In [1]:
import pandas as pd

## Loading data

In [2]:
df = pd.read_csv('./ETHPrize Tagged Data - 20180923.csv')

In [3]:
# View a snippet of the data
df.head(10)

,Name,Who are you and what are you working on?,Topics,Projects,What are your biggest frustrations?,Topics.1,Projects.1,What tools don’t exist at the moment?,Topics.2,Projects.2,...,Projects.10,What are you most excited about in the short term?,Topics.11,Projects.11,Who are the other people you think we should talk to?,Topics.12,Projects.12,Are there any other questions we should be asking?,Topics.13,Projects.13
0,﻿Fabio Berger + Remco Bloemen,0x - Decentralized exchange protocol. It is a ...,"exchange, protocol, standards, transaction, ne...",0x,Getting a simple experimental environment up i...,"experimentation, tracing, profiling, code cove...","Solidity, Remix",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,﻿Leo Logvinov,"Started in blockchain 2 years ago in Berlin, w...","usability, errors","IntelliJ, 0x","Event watching - unreliable, no support for ba...","events, ABI, contracts, debugger, standards, c...","Truffle, Solidity, IntelliJ, Ganache, LLL",Prettier type plugin for solidity. I don’t hav...,NaN,Solidity,...,"Ganache, Solidity, EthereumJS-blockstream",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,﻿Axel Ericsson,I have built 1Protocol\nIt lets smart contract...,"contracts, stake signing, signatures, tokens, ...","MEW, Raiden, 1Protocol",NaN,NaN,NaN,There is no tooling or anything related to sta...,state channels,MEW,...,NaN,Raiden is the golden egg in the space. The exp...,NaN,"Raiden, Ethereum, Counterfactual",NaN,NaN,NaN,NaN,NaN,NaN
3,﻿Mike Goldin,Software developer at Consensys\nKnown for Tok...,"Token Curated Registries, TCRs, design, money,...","Consensys, AdChain",Truffle’s debugger is a bit disappointing. Wor...,"debugger, contracts, proxy, data, production, ...","Truffle, EthPM",Fuzz Testing and formal verification desired.\n,"fuzz testing, formal verification",NaN,...,NaN,Excited for Casper\nApplications implemented i...,state channels,"Casper, Plasma","Infura team, client development\nSpankchain - ...",NaN,"Infura, Spankchain",NaN,NaN,NaN
4,﻿Oleksii,Started working with smart contracts in early ...,"contracts, scalability, tokens, logic, deploym...","Ambisafe, Truffle",Our original vision was to do everything – tes...,"snapshot, deployment, contracts, deployment, t...","built_our_own, Truffle, Testrpc, Remix, Securify",NaN,NaN,NaN,...,"Ethercamp, 4byte.io",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,﻿Brett Sun,Working on Aragon entirely.\nThe end goal is t...,"netowkr, organisation, protocol, money, permis...","Aragon, Ethereum",My biggest general frustration comes with the ...,"security, productivity, ecosystem, best practi...",Ethereum,A nice debugger! Please…\nMore infrastructure ...,"debugger, infrastructure, caching, dapps, events",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,﻿Jorge Izquierdo,Aragon - Decentralized Governance platform\nWe...,"governance, language, natspec, dapps, Ruby, up...",Aragon,NaN,NaN,NaN,NaN,NaN,NaN,...,"Augur, Solidity, EthereumJS-blockstream",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,﻿Jack Peterson and Sparkle,NaN,NaN,NaN,Lack of a debugger - by far the biggest issue....,"debugger, transactions",NaN,Setting breakpoints in tests!\nSalesforce Deve...,"bounties, documentation, gas, yellow paper","Salesforce, Geth, React, IPFS, EVM, Kauri, Eth...",...,"Augur, Raiden, Airbit",Raiden is coming shortly and will be very cool...,NaN,"Raiden, Proof of Steak, uPort, Augur",NaN,NaN,NaN,NaN,NaN,NaN
8,﻿Joey Krug,Co-Chief Investment Officer at Pantera Capital...,NaN,"Pantera Capital, Augur","At the end of the day, it all comes down to th...","scalability, payments, state channels","Ethereum, L4, EOS","My answers changed over a time, 1 year ago it ...",static analysis,NaN,...,"Ethereum, Bitcoin, Geth, parity","In the short term, he’s most excited about Mak...","stability, scalability, sharding",MakerDAO,NaN,NaN,NaN,NaN,NaN,NaN
9,﻿Mark Beylin,Creator of the Bounties Network. Bounties on a...,"bounties, websockets, transactions","Bounties.network, Ethereum, IPFS, Solidity, Me...",Not being able t

## Reformatting the columns for multi-labelling

The topics and projects can be applied to all questions so we can train two common models instead of a
a question specific models, one model for topics and one for projects.

We will also ignore the questions with no answers.

We need to keep the original (line, name, question) to pass from the "natural" representation to the one suitable for
automated multi-labelling.

In summary we will go convert representation as the following

```
   Name  | Q0   T0  P0  Q1    T1  P1        Obs  OldRow Name  Question Answer Topics Projects
0  Alice | foo  a,b  x   oof   b   y  ==>     0    0    Alice  Q0      foo     a,b      x
1  Bob   | bar   b  x,y  rab   c   z          1    0    Alice  Q1      oof       b      y
2  Eve   | baz   a   y   zab   a   x          2    1    Bob    Q0      bar       b    x,y
                                              3    1    Bob    Q1      rab       c      z
                                              4    2    Eve    Q0      baz       a      y
                                              5    2    Eve    Q1      zab       a      x
```

With have 43 columns with the first one being the Name.
First besides the name we need to group them

In [53]:
def getMultiCol(columns):
    length = len(columns)
    qtp = [('Name', '')] # questions, topics, projects
    for col_idx in range(1, length, 3):
        qtp.append((columns[col_idx], 'Answer'))
        qtp.append((columns[col_idx], 'Topics'))
        qtp.append((columns[col_idx], 'Projects'))
    return pd.MultiIndex.from_tuples(qtp, names=('Questions', 'Answers_Topics_Projects'))
        

In [54]:
# Check if we can get what we want
pd.DataFrame(df.drop('Name', axis = 1), columns = getMultiCol(df.columns))

Questions               Name Who are you and what are you working on?         \
Answers_Topics_Projects                                        Answer Topics   
0                        NaN                                      NaN    NaN   
1                        NaN                                      NaN    NaN   
2                        NaN                                      NaN    NaN   
3                        NaN                                      NaN    NaN   
4                        NaN                                      NaN    NaN   
5                        NaN                                      NaN    NaN   
6                        NaN                                      NaN    NaN   
7                        NaN                                      NaN    NaN   
8                        NaN                                      NaN    NaN   
9                        NaN                                      NaN    NaN   
10                       NaN                                      NaN    NaN   
11                       NaN                                      NaN    NaN   
12                       NaN                                      NaN    NaN   
13                       NaN                                      NaN    NaN   
14                       NaN                                      NaN    NaN   
15                       NaN                                      NaN    NaN   
16                       NaN                                      NaN    NaN   
17                       NaN                                      NaN    NaN   
18                       NaN                                      NaN    NaN   
19                       NaN                                      NaN    NaN   
20                       NaN                                      NaN    NaN   
21                       NaN                                      NaN    NaN   
22                       NaN                                      NaN    NaN   
23                       NaN                                      NaN    NaN   
24                       NaN                                      NaN    NaN   
25                       NaN                                      NaN    NaN   
26                       NaN                                      NaN    NaN   
27                       NaN                                      NaN    NaN   
28                       NaN                                      NaN    NaN   
29                       NaN                                      NaN    NaN   
..                       ...                                      ...    ...   
74                       NaN                                      NaN    NaN   
75                       NaN                                      NaN    NaN   
76                       NaN                                      NaN    NaN   
77                       NaN                                      NaN    NaN   
78                       NaN                                      NaN    NaN   
79                       NaN                                      NaN    NaN   
80                       NaN                                      NaN    NaN   
81                       NaN                                      NaN    NaN   
82                       NaN                                      NaN    NaN   
83                       NaN                                      NaN    NaN   
84                       NaN                                      NaN    NaN   
85                       NaN                                      NaN    NaN   
86                       NaN                                      NaN    NaN   
87                       NaN                                      NaN    NaN   
88                       NaN                                      NaN    NaN   
89                       NaN                                      NaN    NaN   
90                       NaN                                      NaN    NaN   


In [57]:
# There is no "set_column" so we transpose back and forth
df = df.T.set_index(getMultiCol(df.columns)).T

In [58]:
# Check
df.head(10)

Questions                                            Name  \
Answers_Topics_Projects                                     
0                          ﻿Fabio Berger + Remco Bloemen    
1                                           ﻿Leo Logvinov   
2                                          ﻿Axel Ericsson   
3                                            ﻿Mike Goldin   
4                                                ﻿Oleksii   
5                                              ﻿Brett Sun   
6                                        ﻿Jorge Izquierdo   
7                              ﻿Jack Peterson and Sparkle   
8                                              ﻿Joey Krug   
9                                            ﻿Mark Beylin   
10                                           ﻿Aidan Hyman   
11                                      ﻿Greg, Stu, David   
12                                         ﻿Jon Maurelian   
13                                           ﻿Nick Dodson   
14                                         ﻿Ilan Benbrith   
15                                            ﻿Goncalo Sa   
16                                 ﻿Fabiano, Jordan, Nick   
17                                         ﻿Andy Milenius   
18                                        ﻿Esteban Ordano   
19                                         ﻿Matias Bargas   
20                                       ﻿Nadav Hollander   
21                                     ﻿Alex van de Sande   
22                                          ﻿Nick Johnson   
23                                      ﻿Elena Nadolinski   
24                                               ﻿Swaroop   
25                       ﻿Alex Beregszazzi & Pawel Bylica   
26                                ﻿Christian Reitwiessner   
27                                   ﻿Fabian Vogelsteller   
28                                        ﻿Hudson Jameson   
29                                         ﻿Martin Swende   
..                                                    ...   
74                                        ﻿Scott Wright     
75                                            ﻿Felix Feng   
76                                        ﻿Simon Jentzsch   
77                                            ﻿Fernando G   
78                                             ﻿Dandelion   
79                                         ﻿David Wolever   
80                                           ﻿James Young   
81                                            ﻿Doug Hoyte   
82                             ﻿Ricardo Guillerme Schmidt   
83                                        ﻿Richard Ramos    
84                                        ﻿Bob Summerwill   
85                         ﻿Yaniv Tal and Brandon Ramirez   
86                                         ﻿Sina Habibian   
87                                          ﻿Truffle Team   
88                                     ﻿Preethi Kasireddy   
89                                        ﻿Alexey Akhunov   
90                                  ﻿Simon de la Rouviere   
91                                               ﻿Alvaro    
92                                         ﻿Javier Franco   
93                                           ﻿Rick Dudley   
94                                         ﻿Evan Van Ness   
95                                      ﻿William Entriken   
96                                      ﻿Christopher Goes   
97                                           ﻿Matt Condon   
98                                         ﻿Ramon Recuero   
99                                        ﻿Facu Spagnuolo   
100                                         ﻿Manuel Araoz   
101                                        ﻿Brendan Chou    
102                                          ﻿Lane Rettig   
103                                                ﻿Kames   

Questions               (Who are you and what are you working on?, Answer)  \
Answers_Topics_Projects                                             Answer   
